# Condition Count Metrics

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/advanced/Condition_Count_Metrics.ipynb)

By default, whylogs tracks several metrics, such as type counts, distribution metrics, cardinality and frequent items. Those are general metrics that are useful for a lot of use cases, but often we need metrics tailored for our application.

__Condition Count Metrics__ gives you the flexibility to define your own customized metrics. It will return the results as counters, which is the number of times the condition was met for a given column. With it, you can define conditions such as regex matching for strings, equalities or inequalities for numerical features, and even define your own function to check for any given condition.

In this example, we will cover:

1. Create metrics for __regex matching__
    - Examples: contains email/credit card number (String features)
2. Create metrics for __(in)equalities__
    - Examples: equal, less, greater, less than, greater than (Numerical features)
3. Combining metrics with __logic operators__ (and, or, not)
    - Examples: Between range, outside of range, not equal (Numerical features)
4. Creating metrics with __custom functions__
    - Examples: is even number, is text preprocessed (Any type)
5. Going Further: Combining this example with other whylogs' features
6. (APPENDIX) Complete code snippets - The complete code snippets (to make it easir to copy and paste)

## Installing whylogs and importing modules

Uncomment the cell below if you don't have whylogs installed yet:

In [1]:
# %pip install whylogs

Let's import all the dependencies for this example upfront:

In [2]:
import pandas as pd

import re
from typing import Any, Dict

import whylogs as why

from whylogs.core.resolvers import Resolver
from whylogs.core.schema import ColumnSchema, DatasetSchema
from whylogs.core.datatypes import DataType

from whylogs.core.metrics import Metric
from whylogs.core.metrics.condition_count_metric import (
    Condition,
    ConditionCountConfig,
    ConditionCountMetric,
)
from whylogs.core.metrics.condition_count_metric import Relation as Rel
from whylogs.core.metrics.condition_count_metric import relation as rel
from whylogs.core.metrics.condition_count_metric import and_relations as and_rel
from whylogs.core.metrics.condition_count_metric import or_relations as or_rel
from whylogs.core.metrics.condition_count_metric import not_relation as not_rel


## 1. Regex Matching

Suppose we have textual columns in our data in which we want to make sure certain elements are present / not present.

For example, for privacy and security issues, we might be interested in tracking the number of times a credit card number appears on a given column, or if we have sensitive email information in another column.

With whylogs, we can define metrics that will count the number of occurences a certain regex pattern is met for a given column.

### Creating sample dataframe

Let's create a simple dataframe.

In this scenario, the `emails` column should have only a valid email, nothing else. As for the `trascriptions` column, we want to make sure existing credit card number was properly masked or removed.

In [3]:
data = {
    "emails": ["my email is my_email_1989@gmail.com","invalidEmail@xyz.toolong","this.is.ok@hotmail.com","not an email"],
    "transcriptions": ["Bob's credit card number is 4000000000000", "Alice's credit card is XXXXXXXXXXXXX", "Hi, my name is Bob", "Hi, I'm Alice"],
}
df = pd.DataFrame(data=data)

whylogs use `Resolvers` in order to define how a column name or data type gets mapped to different metrics.

To define our regex-matching constraints, we will create a CustomResolver. In this example, we want only ConditionCountMetrics to be mapped for every column:


In [4]:
class CustomResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {"condition_count": ConditionCountMetric.zero(column_schema.cfg)}

We now have to define the conditions themselves:

In [5]:
conditions = {
    "containsEmail": Condition(rel(Rel.fullmatch, "[\w.]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}")),
    "containsCreditCard": Condition(rel(Rel.match, ".*4[0-9]{12}(?:[0-9]{3})?"))
}

> Note: The regex expressions are for demonstrational purposes only. These expressions are not general - there will be emails and credit cards whose patterns will not be met by the expression.

In order to define the schema used to track metrics in whylogs, we use a `DatasetSchema`.

In a `DatasetSchema`, we have informations such as the resolver used and extra configurations needed.

We'll pass our custom resolver as our schema's resolver and also pass the conditions we created as part of our schema's configuration, like this:

In [6]:
config = ConditionCountConfig(conditions=conditions)
resolver = CustomResolver()
schema = DatasetSchema(default_configs=config, resolvers=resolver)

Now, we only need to pass our schema when logging our data. Let's also take a look at the metrics, to make sure everythins was tracked correctly:

In [7]:
prof_view = why.log(df, schema=schema).profile().view()
prof_view.to_pandas()

,condition_count/containsCreditCard,condition_count/containsEmail,condition_count/total,type
column,,,,
emails,0,1,4,SummaryType.COLUMN
transcriptions,1,0,4,SummaryType.COLUMN


Let's check the numbers:

For `emails` feature, only one occurence was met for `containsEmail`. That is expected, because the only valid row is the third one ("this.is.ok@hotmail.com"). Others either don't contain an email, are invalid emails or have extra text that are not an email (note we're using `fullmatch` as Relation for the email condition).

For `transcriptions` column, we also have only one match. That is well, since only the first row has a match with the given pattern, and others either don't have a credit card number or are properly "hidden". Note that in this case we want to check for the pattern inside a broader text, so we're using `.*` before the pattern, so the text doesn't have to start with the pattern (whylogs' `Rel.match` uses python's `re.compile().match()` under the hood.)

The available relations for regex matching are the ones used in this example:

- `match`
- `fullmatch`

## 2. Numerical Equalities and Inequalities

For this one, let's create integer and floats columns:

In [8]:
data = {
    "ints": [1,12,42,4],
    "floats": [1.2, 12.3, 42.2, 4.8]

}
df = pd.DataFrame(data=data)

As before, we want to create a `CustomResolver`, set our conditions and pass both to a `DatasetSchema`:

In [9]:
class CustomResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {"condition_count": ConditionCountMetric.zero(column_schema.cfg)}

conditions = {
    "equals42": Condition(rel(Rel.equal, 42)),
    "equals42.2": Condition(rel(Rel.equal, 42.2)),
    "lessthan5": Condition(rel(Rel.less, 5)),
    "morethan40": Condition(rel(Rel.greater, 40)),
}

config = ConditionCountConfig(conditions=conditions)
resolver = CustomResolver()
schema = DatasetSchema(default_configs=config, resolvers=resolver)



Let's log and check the metrics:

In [10]:
prof_view = why.log(df, schema=schema).profile().view()
prof_view.to_pandas()

,condition_count/equals42,condition_count/equals42.2,condition_count/lessthan5,condition_count/morethan40,condition_count/total,type
column,,,,,,
floats,0,1,2,1,4,SummaryType.COLUMN
ints,1,0,2,1,4,SummaryType.COLUMN


We can simply check the original data to verify that the metrics are correct.

We used `equal`, `less` and `greater` in this example, but here's the complete list of available relations:

- `equal` - equal to
- `less` - less than
- `leq` - less than or equal to
- `greater` - greater than
- `geq` - greater than or equal to
- `neq` - not equal to


## 3. Combining metrics with logical operators - AND, OR, NOT

You can also combine relations with logical operators such as __AND__, __OR__ and __NOT__.

Let's stick with the numerical features to show how you can combine relations to assemble conditions such as:

- Value is between a certain range
- Value is outside a certin range
- Value is NOT a certain number

In [11]:
class CustomResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {"condition_count": ConditionCountMetric.zero(column_schema.cfg)}

conditions = {
    "between10and50": Condition(and_rel(rel(Rel.greater, 10), rel(Rel.less, 50))),
    "outside10and50": Condition(or_rel(rel(Rel.less, 10), rel(Rel.greater, 50))),
    "not_42": Condition(not_rel(rel(Rel.equal, 42))),
}

config = ConditionCountConfig(conditions=conditions)
resolver = CustomResolver()
schema = DatasetSchema(default_configs=config, resolvers=resolver)


prof_view = why.log(df, schema=schema).profile().view()
prof_view.to_pandas()


,condition_count/between10and50,condition_count/not_42,condition_count/outside10and50,condition_count/total,type
column,,,,,
floats,2,4,2,4,SummaryType.COLUMN
ints,2,3,2,4,SummaryType.COLUMN


Available logical operators are:
- `and_rel`
- `or_rel`
- `not_rel`

Even though we showed these operators with numerical features, this also works with regex matching conditions shown previously.

## 4. Custom Condition with User-defined functions

If none of the previously conditions are suited to your use case, you are free to define your own custom function to create your own metrics.

Let's see a simple example: suppose we want to check if a certain number is even.

We can define a `is_even` function, as simple as:

In [12]:
def is_even(x: Any) -> bool:
    return x % 2 == 0

And then we proceed as usual, defining our resolver and conditions.

We only have to pass the name of the function to `conditions` as a `Condition` object, like below:

In [13]:
class CustomResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {"condition_count": ConditionCountMetric.zero(column_schema.cfg)}

conditions = {
    "isEven": Condition(is_even),
}

config = ConditionCountConfig(conditions=conditions)
resolver = CustomResolver()
schema = DatasetSchema(default_configs=config, resolvers=resolver)


prof_view = why.log(df, schema=schema).profile().view()
prof_view.to_pandas()


,condition_count/isEven,condition_count/total,type
column,,,
floats,0,4,SummaryType.COLUMN
ints,3,4,SummaryType.COLUMN


### NLP example

For user-defined functions, the sky's the limit for what you can do.

Let's think of another simple cenario for NLP. Suppose our model assumes text to be a certain way. Maybe it was trained and expects:

- lowercased characters
- no digits
- no stopwords

Let's check these conditions for the data below:

In [14]:
data = {
    "trasncriptions": ["I AM BOB AND I LIKE TO SCREAM","i am bob","am alice and am xx years old","am bob and am 42 years old"],
    "ints": [0,1,2,3],
}
df = pd.DataFrame(data=data)

Once again, let's define our function:

In [15]:
def ispreprocessed(x: Any) -> bool:
    stopwords = ["i", "me", "myself"]
    if not isinstance(x, str):
        return False
    
    # should have only lowercase letters and space (no digits)
    if not all(c.islower() or c.isspace() for c in x):
        return False
    # should not contain any words in our stopwords list
    if any(c in stopwords for c in x.split()):
        return False
    return True

> Since this is an example, our `stopwords` list is only a placeholder for the real thing.

The rest is the same as before:

In [16]:
class CustomResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {"condition_count": ConditionCountMetric.zero(column_schema.cfg)}

conditions = {
    "isPreprocessed": Condition(ispreprocessed),
}

config = ConditionCountConfig(conditions=conditions)
resolver = CustomResolver()
schema = DatasetSchema(default_configs=config, resolvers=resolver)


prof_view = why.log(df, schema=schema).profile().view()
prof_view.to_pandas()


,condition_count/isPreprocessed,condition_count/total,type
column,,,
ints,0,4,SummaryType.COLUMN
trasncriptions,1,4,SummaryType.COLUMN


For the `transcriptions` feature, we can see that only the second row is properly preprocessed ("am alice and am xx years old"). The first one contained uppercase characters, the third contained a stopword and the last one contained digits. For the integers column, isPreprocessed returns 0, since it's not a string value. 

## 5. Going Further

You can combine this example with other whylogs' features to cover even more scenarios.

Here are some pointers for some possible use cases:

- I want to track other types of metrics, not only Condition Counts!
    - Check the [Schema Configuration](https://whylogs.readthedocs.io/en/stable/examples/basic/Schema_Configuration.html) example!
- Ok, I have counters. What now?
    - You can set constraints (such as `containsCreditCardNumber` should always be 0). Check the [Metric Constraints](https://whylogs.readthedocs.io/en/stable/examples/basic/Metric_Constraints.html) example!
    - You can store it locally or on S3 for future inspection - Check the [Writing Profiles](https://whylogs.readthedocs.io/en/stable/examples/integrations/writers/Writing_Profiles.html) example!
    - You can send your profiles to a monitoring platform, such as WhyLabs - Check the [Writing Profiles to WhyLabs](https://whylogs.readthedocs.io/en/stable/examples/integrations/writers/Writing_to_WhyLabs.html) example!

## Appendix - Complete Code Snippets

Here are the complete code snippets - just to make it easier to copy/paste!

### Regex example

In [17]:
import pandas as pd

import re
from typing import Any, Dict

import whylogs as why

from whylogs.core.resolvers import Resolver
from whylogs.core.schema import ColumnSchema, DatasetSchema
from whylogs.core.datatypes import DataType

from whylogs.core.metrics import Metric
from whylogs.core.metrics.condition_count_metric import (
    Condition,
    ConditionCountConfig,
    ConditionCountMetric,
)
from whylogs.core.metrics.condition_count_metric import Relation as Rel
from whylogs.core.metrics.condition_count_metric import relation as rel

data = {
    "emails": ["my email is my_email_1989@gmail.com","invalidEmail@xyz.toolong","this.is.ok@hotmail.com","not an email"],
    "transcriptions": ["Bob's credit card number is 4000000000000", "Alice's credit card is XXXXXXXXXXXXX", "Hi, my name is Bob", "Hi, I'm Alice"],
}
df = pd.DataFrame(data=data)

class CustomResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {"condition_count": ConditionCountMetric.zero(column_schema.cfg)}

conditions = {
    "containsEmail": Condition(rel(Rel.fullmatch, "[\w.]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}")),
    "containsCreditCard": Condition(rel(Rel.match, ".*4[0-9]{12}(?:[0-9]{3})?"))
}

config = ConditionCountConfig(conditions=conditions)
resolver = CustomResolver()
schema = DatasetSchema(default_configs=config, resolvers=resolver)

prof_view = why.log(df, schema=schema).profile().view()
prof_view.to_pandas()

,condition_count/containsCreditCard,condition_count/containsEmail,condition_count/total,type
column,,,,
emails,0,1,4,SummaryType.COLUMN
transcriptions,1,0,4,SummaryType.COLUMN


### Equalities Example

In [18]:
import pandas as pd

import re
from typing import Any, Dict

import whylogs as why

from whylogs.core.resolvers import Resolver
from whylogs.core.schema import ColumnSchema, DatasetSchema
from whylogs.core.datatypes import DataType

from whylogs.core.metrics import Metric
from whylogs.core.metrics.condition_count_metric import (
    Condition,
    ConditionCountConfig,
    ConditionCountMetric,
)
from whylogs.core.metrics.condition_count_metric import Relation as Rel
from whylogs.core.metrics.condition_count_metric import relation as rel

data = {
    "ints": [1,12,42,4],
    "floats": [1.2, 12.3, 42.2, 4.8]

}
df = pd.DataFrame(data=data)

class CustomResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {"condition_count": ConditionCountMetric.zero(column_schema.cfg)}

conditions = {
    "equals42": Condition(rel(Rel.equal, 42)),
    "equals42.2": Condition(rel(Rel.equal, 42.2)),
    "lessthan5": Condition(rel(Rel.less, 5)),
    "morethan40": Condition(rel(Rel.greater, 40)),
}

config = ConditionCountConfig(conditions=conditions)
resolver = CustomResolver()
schema = DatasetSchema(default_configs=config, resolvers=resolver)

prof_view = why.log(df, schema=schema).profile().view()
prof_view.to_pandas()

,condition_count/equals42,condition_count/equals42.2,condition_count/lessthan5,condition_count/morethan40,condition_count/total,type
column,,,,,,
floats,0,1,2,1,4,SummaryType.COLUMN
ints,1,0,2,1,4,SummaryType.COLUMN


### Logical Operators Example

In [19]:
import pandas as pd

import re
from typing import Any, Dict

import whylogs as why

from whylogs.core.resolvers import Resolver
from whylogs.core.schema import ColumnSchema, DatasetSchema
from whylogs.core.datatypes import DataType

from whylogs.core.metrics import Metric
from whylogs.core.metrics.condition_count_metric import (
    Condition,
    ConditionCountConfig,
    ConditionCountMetric,
)
from whylogs.core.metrics.condition_count_metric import Relation as Rel
from whylogs.core.metrics.condition_count_metric import relation as rel

from whylogs.core.metrics.condition_count_metric import and_relations as and_rel
from whylogs.core.metrics.condition_count_metric import or_relations as or_rel
from whylogs.core.metrics.condition_count_metric import not_relation as not_rel

data = {
    "ints": [1,12,42,4],
    "floats": [1.2, 12.3, 42.2, 4.8]

}
df = pd.DataFrame(data=data)

class CustomResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {"condition_count": ConditionCountMetric.zero(column_schema.cfg)}

conditions = {
    "between10and50": Condition(and_rel(rel(Rel.greater, 10), rel(Rel.less, 50))),
    "outside10and50": Condition(or_rel(rel(Rel.less, 10), rel(Rel.greater, 50))),
    "not_42": Condition(not_rel(rel(Rel.equal, 42))),
}

config = ConditionCountConfig(conditions=conditions)
resolver = CustomResolver()
schema = DatasetSchema(default_configs=config, resolvers=resolver)


prof_view = why.log(df, schema=schema).profile().view()
prof_view.to_pandas()


,condition_count/between10and50,condition_count/not_42,condition_count/outside10and50,condition_count/total,type
column,,,,,
floats,2,4,2,4,SummaryType.COLUMN
ints,2,3,2,4,SummaryType.COLUMN


### User-defined function  - isEven

In [20]:
import pandas as pd

import re
from typing import Any, Dict

import whylogs as why

from whylogs.core.resolvers import Resolver
from whylogs.core.schema import ColumnSchema, DatasetSchema
from whylogs.core.datatypes import DataType

from whylogs.core.metrics import Metric
from whylogs.core.metrics.condition_count_metric import (
    Condition,
    ConditionCountConfig,
    ConditionCountMetric,
)
from whylogs.core.metrics.condition_count_metric import Relation as Rel
from whylogs.core.metrics.condition_count_metric import relation as rel

def is_even(x: Any) -> bool:
    return x % 2 == 0

def ispreprocessed(x: Any) -> bool:
    stopwords = ["i", "me", "myself"]
    if not isinstance(x, str):
        return False
    
    # should have only lowercase letters and space (no digits)
    if not all(c.islower() or c.isspace() for c in x):
        return False
    # should not contain any words in our stopwords list
    if any(c in stopwords for c in x.split()):
        return False
    return True

data = {
    "trasncriptions": ["I AM BOB AND I LIKE TO SCREAM","i am bob","am alice and am xx years old","am bob and am 42 years old"],
    "ints": [1,12,42,4],
    "floats": [1.2, 12.3, 42.2, 4.8]
}
df = pd.DataFrame(data=data)

class CustomResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {"condition_count": ConditionCountMetric.zero(column_schema.cfg)}

conditions = {
    "isPreprocessed": Condition(ispreprocessed),
    "isEven": Condition(is_even)
}

config = ConditionCountConfig(conditions=conditions)
resolver = CustomResolver()
schema = DatasetSchema(default_configs=config, resolvers=resolver)


prof_view = why.log(df, schema=schema).profile().view()
prof_view.to_pandas()


,condition_count/isEven,condition_count/isPreprocessed,condition_count/total,type
column,,,,
floats,0,0,4,SummaryType.COLUMN
ints,3,0,4,SummaryType.COLUMN
trasncriptions,0,1,4,SummaryType.COLUMN
